<a href="https://colab.research.google.com/github/gauravm999/CarPark_Revenue_Prediction/blob/main/Detecting_Sales_Opportunities_MSTeams_AI_Notes_%26_Agentic_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Detecting Sales Opportunities from Microsoft Teams AI Notes & Agentic RAG

Step 1: Setup

In [ ]:
import re
import random
from transformers import pipeline

# Load a pre-trained NLP pipeline for text classification
# Here we can use a pipeline like Huggingface's zero-shot-classification model.
classifier = pipeline("zero-shot-classification")

# Define categories for sales opportunities
lead_categories = [
    "budget approval",
    "interest in AI solutions",
    "vendor evaluation",
    "purchase decision",
    "business opportunity",
    "request for proposal"
]

# Simulated conversation data from Microsoft Teams
conversation_data = [
    {"speaker": "Client", "text": "We are looking into allocating the budget for AI next quarter."},
    {"speaker": "Client", "text": "Could you provide us a proposal on predictive analytics?"},
    {"speaker": "Sales Rep", "text": "Sure, we can help you evaluate vendors and determine the best approach."},
    {"speaker": "Client", "text": "We want to improve our supply chain with automation."},
]

No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Step 2: Analyze Conversation to Detect Sales Leads
Here, the AI analyzes the conversation using a pre-trained NLP model to classify the text as potential sales-related opportunities.

In [ ]:
# Function to identify sales leads using text classification
def identify_sales_opportunities(conversation, classifier, lead_categories):
    sales_opportunities = []

    for message in conversation:
        speaker = message["speaker"]
        text = message["text"]

        # Classify the text into one of the predefined sales categories
        result = classifier(text, lead_categories)

        # Extract the most likely category
        most_likely_category = result["labels"][0]
        score = result["scores"][0]

        # Consider only high-confidence opportunities (adjust threshold as needed)
        if score > 0.7:
            sales_opportunities.append({
                "speaker": speaker,
                "text": text,
                "lead_category": most_likely_category,
                "confidence": score
            })

    return sales_opportunities


Step 3: Generate Follow-up Actions
Once a lead or sales opportunity is identified, the AI generates relevant follow-up actions.

In [ ]:
# Function to generate follow-up actions based on identified leads
def generate_followups(sales_opportunities):
    followup_actions = []

    for opportunity in sales_opportunities:
        lead_category = opportunity["lead_category"]
        speaker = opportunity["speaker"]

        # Generate dynamic responses based on the lead category
        if lead_category == "budget approval":
            followup_actions.append(f"Follow up with {speaker} about their budget allocation for the next quarter.")
        elif lead_category == "interest in AI solutions":
            followup_actions.append(f"Schedule a demo of our AI solutions with {speaker}.")
        elif lead_category == "request for proposal":
            followup_actions.append(f"Send a detailed proposal to {speaker} on predictive analytics solutions.")
        elif lead_category == "business opportunity":
            followup_actions.append(f"Discuss business opportunities with {speaker} in automation.")

    return followup_actions


Step 4: Full Pipeline: Detect Sales Leads and Generate Follow-ups
This is how the AI would detect opportunities from the conversation and suggest actions:

In [ ]:
# Full pipeline: detecting sales leads and generating follow-ups
sales_opportunities = identify_sales_opportunities(conversation_data, classifier, lead_categories)
followup_actions = generate_followups(sales_opportunities)

# Output results
print("Identified Sales Opportunities:\n")
for opportunity in sales_opportunities:
    print(f"Speaker: {opportunity['speaker']}")
    print(f"Text: {opportunity['text']}")
    print(f"Category: {opportunity['lead_category']} (Confidence: {opportunity['confidence']:.2f})\n")

print("Generated Follow-up Actions:\n")
for action in followup_actions:
    print(action)


Identified Sales Opportunities:

Speaker: Client
Text: We are looking into allocating the budget for AI next quarter.
Category: interest in AI solutions (Confidence: 0.73)

Speaker: Client
Text: Could you provide us a proposal on predictive analytics?
Category: request for proposal (Confidence: 0.87)

Speaker: Sales Rep
Text: Sure, we can help you evaluate vendors and determine the best approach.
Category: vendor evaluation (Confidence: 0.86)

Speaker: Client
Text: We want to improve our supply chain with automation.
Category: business opportunity (Confidence: 0.70)

Generated Follow-up Actions:

Schedule a demo of our AI solutions with Client.
Send a detailed proposal to Client on predictive analytics solutions.
Discuss business opportunities with Client in automation.


# Agentic RAG for Sales Lead Identification

Step 1: Setup – Adding RAG Retrieval for External Knowledge

In [ ]:
from transformers import pipeline

# Load a pre-trained question-answering pipeline
# This will act as the retrieval mechanism for relevant external knowledge
retriever = pipeline("question-answering")

# Example external knowledge source (could be CRM, database, etc.)
external_knowledge = """
Our AI solutions include predictive analytics, customer segmentation, and supply chain optimization.
We have worked with large CPG clients such as Procter & Gamble and Unilever.
Our product pricing varies based on scale, with custom models for enterprise clients.
We offer post-implementation support and training services for large AI solution deployments.
"""

# Modified lead categories to interact with external knowledge
lead_categories = [
    "budget approval",
    "interest in AI solutions",
    "vendor evaluation",
    "purchase decision",
    "business opportunity",
    "request for proposal"
]

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Step 2: Function to Retrieve External Knowledge Based on Sales Opportunity
This function will trigger the retrieval of relevant information when a sales opportunity is detected.

In [ ]:
# Function to retrieve relevant knowledge based on detected sales lead
def retrieve_knowledge(query, external_knowledge, retriever):
    response = retriever(question=query, context=external_knowledge)
    return response["answer"]

Step 3: Agent Module to Generate Context-Aware Responses
Once relevant knowledge is retrieved, the AI agent generates contextual suggestions or responses.

In [ ]:
# Function to generate personalized follow-up actions based on retrieved knowledge
def generate_agentic_responses(sales_opportunities, external_knowledge, retriever):
    agentic_responses = []

    for opportunity in sales_opportunities:
        lead_category = opportunity["lead_category"]
        speaker = opportunity["speaker"]
        text = opportunity["text"]

        # Dynamically retrieve knowledge based on detected sales lead
        if lead_category == "interest in AI solutions":
            query = "What AI solutions do we offer for CPG clients?"
            relevant_info = retrieve_knowledge(query, external_knowledge, retriever)
            agentic_responses.append(f"{speaker} mentioned interest in AI solutions. You could mention: {relevant_info}")

        elif lead_category == "request for proposal":
            query = "What pricing models do we offer for AI solutions?"
            relevant_info = retrieve_knowledge(query, external_knowledge, retriever)
            agentic_responses.append(f"{speaker} requested a proposal. Mention: {relevant_info}")

        elif lead_category == "business opportunity":
            query = "What support services do we offer after AI solution deployment?"
            relevant_info = retrieve_knowledge(query, external_knowledge, retriever)
            agentic_responses.append(f"{speaker} mentioned a business opportunity. You could highlight: {relevant_info}")

        # Other lead categories can be extended with more external knowledge queries.

    return agentic_responses # This line was not indented correctly

Step 4: Full Pipeline with Agentic RAG for Sales Lead Identification
This pipeline will detect sales opportunities, retrieve relevant information, and generate agentic responses.

In [ ]:
# Full pipeline: Detecting sales leads, retrieving knowledge, and generating agentic responses
sales_opportunities = identify_sales_opportunities(conversation_data, classifier, lead_categories)
agentic_responses = generate_agentic_responses(sales_opportunities, external_knowledge, retriever)

# Output results
print("Identified Sales Opportunities with Agentic Responses:\n")
for opportunity in sales_opportunities:
    print(f"Speaker: {opportunity['speaker']}")
    print(f"Text: {opportunity['text']}")
    print(f"Category: {opportunity['lead_category']} (Confidence: {opportunity['confidence']:.2f})\n")

print("Generated Agentic Responses:\n")
for response in agentic_responses:
    print(response)

Identified Sales Opportunities with Agentic Responses:

Speaker: Client
Text: We are looking into allocating the budget for AI next quarter.
Category: interest in AI solutions (Confidence: 0.73)

Speaker: Client
Text: Could you provide us a proposal on predictive analytics?
Category: request for proposal (Confidence: 0.87)

Speaker: Sales Rep
Text: Sure, we can help you evaluate vendors and determine the best approach.
Category: vendor evaluation (Confidence: 0.86)

Speaker: Client
Text: We want to improve our supply chain with automation.
Category: business opportunity (Confidence: 0.70)

Generated Agentic Responses:

Client mentioned interest in AI solutions. You could mention: predictive analytics, customer segmentation, and supply chain optimization
Client requested a proposal. Mention: custom models
Client mentioned a business opportunity. You could highlight: post-implementation support and training services
